In [5]:
import os
import fitz  # PyMuPDF
import pandas as pd
import re # 정규 표현식 모듈
import csv

# 이전에 정의했던 PDF 파일 목록 가져오는 함수
def list_pdf_files(directory_path):
    """
    지정된 디렉토리에서 PDF 파일 목록을 반환합니다.
    """
    pdf_files = []
    if not os.path.isdir(directory_path):
        print(f"오류: '{directory_path}'는 유효한 디렉토리가 아닙니다.")
        return pdf_files
    for filename in os.listdir(directory_path):
        if filename.lower().endswith(".pdf") and \
           os.path.isfile(os.path.join(directory_path, filename)):
            pdf_files.append(filename)
    return pdf_files

# 텍스트 추출 함수 (이전과 동일)
def extract_text_from_pdf(file_path):
    """
    주어진 PDF 파일 경로에서 텍스트를 추출합니다.
    """
    try:
        doc = fitz.open(file_path)
        full_text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            full_text += page.get_text("text")
        doc.close()
        return full_text
    except Exception as e:
        print(f"파일 '{file_path}' 처리 중 오류: {e}")
        return None

# PDF 파일 경로 설정 및 목록 가져오기
base_directory = "/Users/jaesolshin/Documents/GitHub/HW_analysis/알리오"
pdf_files = list_pdf_files(base_directory)

if not pdf_files:
    print("알리오 폴더에 PDF 파일이 없습니다.")
else:
    # 첫 번째 PDF 파일 내용 확인 (개발 및 테스트용)
    test_pdf_path = os.path.join(base_directory, pdf_files[0]) # (재)우체국금융개발원...pdf
    print(f"--- 다음 파일의 텍스트 내용을 추출합니다: {test_pdf_path} ---")
    test_text_content = extract_text_from_pdf(test_pdf_path)
    if test_text_content:
        print(test_text_content)
    else:
        print("텍스트를 추출하지 못했습니다.")


--- 다음 파일의 텍스트 내용을 추출합니다: /Users/jaesolshin/Documents/GitHub/HW_analysis/알리오/(재)우체국금융개발원_노동조합+가입정보.pdf ---
15-1. 노동조합 가입정보
(기본 현황)
(2025년 1/4분기)
(재)우체국금융개발원
노동조합 가입정보 내역
단일노조
노동조합 명칭
우체국금융개발원노동조합
노동조합 설립일
2000년 09월 25일
위원장
성명
O O O
임기
2023.03.01 ~ 2026.02.28
노동조합 가입범위
"4급 및 선임연구원 이상으로 팀장 이상 보직이 있는자", "인사, 노무, 회계,
감사 업무를 수행하는 자", "임원 비서 및 전용 운전원", "정보보호팀 전산
담당자 중 인사 평가 시스템 접근 관리 업무를 수행하는 자"를 제외한 전
직원
가입대상 인원
742 명
조합원수
정규직(일반정규직)
188 명
비정규직
9 명
정규직(무기계약직)
196 명
교섭권 여부
있음 / 교섭대표노조
근로시간면제 체결내용
시간
5,000 시간
풀타임
2 명
파트타임
4 명
전임자수
무급
0 명
상급단체
총연합단체
한국노총
연합단체
전국공공산업노동조합연맹
<참고> 노동조합 업무부서 및 담당자
이름
부서명
직책
전화번호
이진우
경영지원실 운영지원팀
과장
02-2610-6065
기준일
2025년 03월 31일
제출일
2025년 04월 03일
기관 공시 담당자
구분
담당자명
부서명
전화번호
작성자
이진우
경영지원실 운영지원팀
02-2610-6065
감독자
조평제
전략기획실
02-2610-6010
확인자
이은주
감사실
02-2610-6180



In [6]:
import os
import fitz  # PyMuPDF
import pandas as pd
import re
import csv

# --- Helper Functions (동일) ---
def search_value(pattern, text, group_index=1, default_value=None, strip_chars=None, flags=0):
    """텍스트에서 정규식 패턴에 맞는 값을 찾아 반환, 특정 문자 제거 기능 포함."""
    if not text: return default_value
    match = re.search(pattern, text, flags)
    if match:
        value = match.group(group_index)
        if value is not None:
            value = value.strip()
            if strip_chars:
                for char_to_strip in strip_chars:
                    value = value.replace(char_to_strip, "")
                value = value.strip()
            if value == '-' or value == '':
                return default_value
            return value
    return default_value

def search_block_content(start_keyword, end_keyword_pattern, text):
    """시작 키워드와 종료 키워드(패턴) 사이의 텍스트 블록을 추출."""
    if not text: return None
    pattern = re.compile(f"{re.escape(start_keyword)}\\n?([\\s\\S]*?)(?=\\n(?:{end_keyword_pattern}|$))", re.MULTILINE | re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(1).strip()
    return None

# --- Main Script ---

# 스키마 (교섭대표노조_여부 포함)
schema = [
    "출처_파일명", "기업_구분", "노조_순번", "노동조합_명칭", "노동조합_설립일", "위원장_성명", "위원장_임기",
    "노동조합_가입범위", "가입대상_인원", "조합원수_정규직_일반정규직", "조합원수_비정규직",
    "조합원수_정규직_무기계약직", "교섭권_여부", "교섭대표노조_여부", "근로시간면제_시간",
    "근로시간면제_풀타임_인원", "근로시간면제_파트타임_인원", "전임자수_무급_인원",
    "상급단체_총연합단체", "상급단체_연합단체",
    "업무담당자_이름", "업무담당자_부서명", "업무담당자_직책", "업무담당자_전화번호",
    "기준일", "제출일",
    "공시_작성자_담당자명", "공시_작성자_부서명", "공시_작성자_전화번호",
    "공시_감독자_담당자명", "공시_감독자_부서명", "공시_감독자_전화번호",
    "공시_확인자_담당자명", "공시_확인자_부서명", "공시_확인자_전화번호",
]

all_unions_data = []

if 'pdf_files' not in locals():
     print("pdf_files 리스트가 정의되지 않았습니다. 첫 번째 코드 셀을 먼저 실행해주세요.")
else:
    for pdf_filename in pdf_files:
        pdf_full_path = os.path.join(base_directory, pdf_filename)
        full_pdf_text = extract_text_from_pdf(pdf_full_path)

        if not full_pdf_text:
            print(f"'{pdf_filename}'에서 텍스트를 추출하지 못해 건너<0xEB><0xA9><0xB5>니다.")
            continue

        is_multiple_union_file = bool(re.search(r"복수노조 / 제\d+노조", full_pdf_text))
        file_type = "복수노조" if is_multiple_union_file else "단일노조"

        union_sections = []
        if is_multiple_union_file:
            split_pattern = r"(복수노조 / 제(\d+)노조)"
            parts = re.split(split_pattern, full_pdf_text)
            current_header_info = None
            current_block_texts = []
            for part in parts:
                 match = re.match(split_pattern, part)
                 if match:
                     if current_header_info and current_block_texts:
                         union_sections.append({"header": current_header_info[0], "number": current_header_info[1], "text": "".join(current_block_texts)})
                     current_header_info = (part.strip(), match.group(2))
                     current_block_texts = []
                 else:
                     if current_header_info:
                         current_block_texts.append(part)
            if current_header_info and current_block_texts:
                union_sections.append({"header": current_header_info[0], "number": current_header_info[1], "text": "".join(current_block_texts)})
        else:
            union_sections.append({"header": "단일노조", "number": None, "text": full_pdf_text})

        if not union_sections and full_pdf_text:
             union_sections.append({"header": "단일노조 (추정)", "number": None, "text": full_pdf_text})

        # --- 각 노조 섹션별로 정보 추출 ---
        for section in union_sections:
            current_union_text_block = section["text"]
            union_sequence_number = section["number"]

            if not current_union_text_block.strip(): continue

            union_data = {field: None for field in schema}
            union_data["출처_파일명"] = pdf_filename
            union_data["기업_구분"] = file_type
            union_data["노조_순번"] = union_sequence_number

            # 필드 추출
            union_data["노동조합_명칭"] = search_value(r"노동조합 명칭\n([^\n]+)", current_union_text_block)
            union_data["노동조합_설립일"] = search_value(r"노동조합 설립일\n([^\n]+)", current_union_text_block)
            chairman_block = search_block_content(r"위원장", r"노동조합 가입범위|가입대상 인원", current_union_text_block)
            if chairman_block:
                union_data["위원장_성명"] = search_value(r"성명\n([^\n]+)", chairman_block)
                union_data["위원장_임기"] = search_value(r"임기\n([^\n]+)", chairman_block)
            union_data["노동조합_가입범위"] = search_block_content(r"노동조합 가입범위", r"가입대상 인원", current_union_text_block)
            union_data["가입대상_인원"] = search_value(r"가입대상 인원\n([^\n]+)", current_union_text_block, strip_chars=["명", ","])
            membership_block = search_block_content(r"조합원수", r"교섭권 여부", current_union_text_block)
            if membership_block:
                union_data["조합원수_정규직_일반정규직"] = search_value(r"정규직\(일반정규직\)\n([^\n]+)", membership_block, strip_chars=["명", ","])
                union_data["조합원수_비정규직"] = search_value(r"^비정규직\n([^\n]+)$", membership_block, strip_chars=["명", ","], flags=re.MULTILINE)
                union_data["조합원수_정규직_무기계약직"] = search_value(r"정규직\(무기계약직\)\n([^\n]+)", membership_block, strip_chars=["명", ","])

            # 교섭권 여부 추출 및 정제
            bargaining_text_raw = search_value(r"교섭권 여부\n([^\n]+)", current_union_text_block)
            if bargaining_text_raw:
                 # " / 교섭대표노조" 또는 유사 패턴 제거
                 union_data["교섭권_여부"] = re.sub(r"\s*/\s*교섭대표노조.*$", "", bargaining_text_raw).strip()
                 # 교섭대표노조 여부 판단
                 if "교섭대표노조" in bargaining_text_raw:
                     union_data["교섭대표노조_여부"] = "Y"
                 else:
                     union_data["교섭대표노조_여부"] = "N"
            else:
                union_data["교섭권_여부"] = None
                union_data["교섭대표노조_여부"] = None # 기본값 None 유지

            time_exemption_block = search_block_content(r"근로시간면제 (?:체결내용|체결내역)", r"전임자수", current_union_text_block)
            if time_exemption_block:
                union_data["근로시간면제_시간"] = search_value(r"시간\n([^\n]+)", time_exemption_block, strip_chars=["시간", ","])
                union_data["근로시간면제_풀타임_인원"] = search_value(r"풀타임\n([^\n]+)", time_exemption_block, strip_chars=["명", ","])
                union_data["근로시간면제_파트타임_인원"] = search_value(r"파트타임\n([^\n]+)", time_exemption_block, strip_chars=["명", ","])
            full_time_officer_block = search_block_content(r"전임자수", r"상급단체", current_union_text_block)
            if full_time_officer_block:
                 union_data["전임자수_무급_인원"] = search_value(r"무급\n([^\n]+)", full_time_officer_block, strip_chars=["명", ","])
            higher_org_block_end_pattern = r"<참고>|기관 세부 작성기준|기준일|기관 공시 담당자|복수노조 / 제\d+노조|총 조합원수|동시\s*가입자수"
            higher_org_block = search_block_content(r"상급단체", higher_org_block_end_pattern, current_union_text_block)
            if higher_org_block:
                union_data["상급단체_총연합단체"] = search_value(r"총연합단체\n([^\n]+)", higher_org_block)
                union_data["상급단체_연합단체"] = search_value(r"연합단체\n([^\n]+)", higher_org_block)

            # 공통 정보 (문서 전체에서 찾음)
            duty_officer_block_end_pattern = r"기관 세부 작성기준|기준일|기관 공시 담당자"
            duty_officer_block = search_block_content(r"<참고> 노동조합 업무부서 및 담당자", duty_officer_block_end_pattern, full_pdf_text)
            if duty_officer_block:
                 duty_officer_match = re.search(r"이름\s*부서명\s*직책\s*전화번호\s*\n([^\n]+)\s+([^\n]+)\s+([^\n]+)\s+([^\n]+)", duty_officer_block, re.MULTILINE)
                 if duty_officer_match:
                    union_data["업무담당자_이름"] = duty_officer_match.group(1).strip()
                    union_data["업무담당자_부서명"] = duty_officer_match.group(2).strip()
                    union_data["업무담당자_직책"] = duty_officer_match.group(3).strip()
                    union_data["업무담당자_전화번호"] = duty_officer_match.group(4).strip()
            union_data["기준일"] = search_value(r"^기준일\n([^\n]+)", full_pdf_text, flags=re.MULTILINE)
            union_data["제출일"] = search_value(r"^제출일\n([^\n]+)", full_pdf_text, flags=re.MULTILINE)
            disclosure_officer_block = search_block_content(r"기관 공시 담당자", r"^\s*$", full_pdf_text)
            if disclosure_officer_block:
                writer_match = re.search(r"작성자\s+([^\n]+)\s+([^\n]+)\s+([\d-]+)", disclosure_officer_block)
                if writer_match:
                    union_data["공시_작성자_담당자명"] = writer_match.group(1).strip()
                    union_data["공시_작성자_부서명"] = writer_match.group(2).strip()
                    union_data["공시_작성자_전화번호"] = writer_match.group(3).strip()
                supervisor_match = re.search(r"감독자\s+([^\n]+)\s+([^\n]+)\s+([\d-]+)", disclosure_officer_block)
                if supervisor_match:
                    union_data["공시_감독자_담당자명"] = supervisor_match.group(1).strip()
                    union_data["공시_감독자_부서명"] = supervisor_match.group(2).strip()
                    union_data["공시_감독자_전화번호"] = supervisor_match.group(3).strip()
                checker_match = re.search(r"확인자\s+([^\n]+)\s+([^\n]+)\s+([\d-]+)", disclosure_officer_block)
                if checker_match:
                    union_data["공시_확인자_담당자명"] = checker_match.group(1).strip()
                    union_data["공시_확인자_부서명"] = checker_match.group(2).strip()
                    union_data["공시_확인자_전화번호"] = checker_match.group(3).strip()

            all_unions_data.append(union_data)

# 추출된 데이터를 Pandas DataFrame으로 변환하여 확인
df_unions = pd.DataFrame(all_unions_data, columns=schema)
print("\n--- 추출된 노조 정보 (교섭대표노조 분리 적용) ---")
print(df_unions.to_string())

# CSV 파일로 저장
output_csv_filename = "노동조합_가입정보_추출결과.csv" # 파일명 유지
output_csv_path = os.path.join(base_directory, output_csv_filename)
df_unions.to_csv(output_csv_path, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_NONNUMERIC)
print(f"\n--- CSV 파일 저장 완료: {output_csv_path} ---")


--- 추출된 노조 정보 (교섭대표노조 분리 적용) ---
                                                     출처_파일명 기업_구분 노조_순번          노동조합_명칭       노동조합_설립일 위원장_성명                   위원장_임기                                                                                                                                                                                                                                                                                                              노동조합_가입범위 가입대상_인원 조합원수_정규직_일반정규직 조합원수_비정규직 조합원수_정규직_무기계약직     교섭권_여부 교섭대표노조_여부 근로시간면제_시간 근로시간면제_풀타임_인원 근로시간면제_파트타임_인원 전임자수_무급_인원 상급단체_총연합단체 상급단체_연합단체 업무담당자_이름    업무담당자_부서명 업무담당자_직책    업무담당자_전화번호            기준일            제출일 공시_작성자_담당자명   공시_작성자_부서명   공시_작성자_전화번호 공시_감독자_담당자명 공시_감독자_부서명   공시_감독자_전화번호 공시_확인자_담당자명 공시_확인자_부서명   공시_확인자_전화번호
0       (재)우체국금융개발원_노동조합+가입정보.pdf  단일노조  None     우체국금융개발원노동조합  2000년 09월 25일  O O O  2023.03.01 ~ 2026.02.28                                                        

In [10]:

# CSV 파일로 저장
output_csv_filename = "노동조합_가입정보_추출결과.csv" # 파일명 유지
output_csv_path = os.path.join(base_directory, output_csv_filename)
df_unions.to_csv(output_csv_path, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_NONNUMERIC)
print(f"\n--- CSV 파일 저장 완료: {output_csv_path} ---")


--- CSV 파일 저장 완료: /Users/jaesolshin/Documents/GitHub/HW_analysis/알리오/노동조합_가입정보_추출결과.csv ---


In [7]:
df_unions.iloc[1,].to_string

<bound method Series.to_string of 출처_파일명            (재)우체국물류지원단_노동조합+가입정보...
기업_구분                                                          복수노조
노조_순번                                                             1
노동조합_명칭                                               우체국물류지원단 노동조합
노동조합_설립일                                              1981년 08월 04일
위원장_성명                                                          고돈섭
위원장_임기                                      2023.05.01 ~ 2026.04.30
노동조합_가입범위         ① 본 협약에서 조합원이라 함은 노동조합 및 노동관계조정법에서 정한 사\n용자를 제...
가입대상_인원                                                        1714
조합원수_정규직_일반정규직                                                  489
조합원수_비정규직                                                       138
조합원수_정규직_무기계약직                                                  311
교섭권_여부                                                           있음
교섭대표노조_여부                                                         Y
근로시간면제_시간     

In [8]:
# Excel 파일로 저장
output_excel_filename = "노동조합_가입정보_추출결과.xlsx" # Excel 파일명
output_excel_path = os.path.join(base_directory, output_excel_filename)

# df_unions.to_excel() 메소드를 사용하여 저장
# index=False: DataFrame의 인덱스를 파일에 쓰지 않음
# engine='openpyxl': Excel 파일을 쓰기 위한 엔진 지정 (필요시 설치: pip install openpyxl)
try:
    df_unions.to_excel(output_excel_path, index=True, engine='openpyxl')
    print(f"\n--- Excel 파일 저장 완료: {output_excel_path} ---")
except ImportError:
    print("\n--- Excel 저장을 위해 'openpyxl' 라이브러리가 필요합니다. ---")
    print("터미널에서 'pip install openpyxl' 명령어를 실행하여 설치해주세요.")
except Exception as e:
    print(f"\n--- Excel 파일 저장 중 오류 발생: {e} ---")


--- Excel 파일 저장 완료: /Users/jaesolshin/Documents/GitHub/HW_analysis/알리오/노동조합_가입정보_추출결과.xlsx ---
